## Reference Notebook - https://www.kaggle.com/julian3833/reef-starter-torch-fasterrcnn-infer-lb-0-416/notebook

In [ ]:
import numpy as np
import pandas as pd
from albumentations.pytorch.transforms import ToTensorV2

import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
WEIGHTS_FILE = "../input/cots-torch-fasterrcnn-weights/fasterrcnn_resnet50_fpn-e10.bin"

## Load Model

In [ ]:
def get_model():
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=False)
    num_classes = 2  # 1 class (starfish) + background

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features

    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # Load the trained weights
    model.load_state_dict(torch.load(WEIGHTS_FILE))
    model.eval()

    model = model.to(device)
    return model

model = get_model()

## Predict

In [ ]:
detection_threshold = 0.5

In [ ]:
def format_prediction_string(boxes, scores):
    # Format as specified in the evaluation page
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.2f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))

    return " ".join(pred_strings)


def predict(model, pixel_array):
    # Predictions for a single image
    
    # Apply all the transformations that are required
    pixel_array = pixel_array.astype(np.float32) / 255.
    tensor_img = ToTensorV2(p=1.0)(image=pixel_array)['image'].unsqueeze(0)
    
    # Get predictions
    with torch.no_grad():
        outputs = model(tensor_img.to(device))[0]
    
    # Move predictions to cpu and numpy
    boxes = outputs['boxes'].data.cpu().numpy()
    scores = outputs['scores'].data.cpu().numpy()
    
    # Filter predictions with low score
    boxes = boxes[scores >= detection_threshold].astype(np.int32)
    scores = scores[scores >= detection_threshold]
    
    # Go back from x_min, y_min, x_max, y_max to x_min, y_min, w, h
    boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
    boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
  
    # Format results as requested in the Evaluation tab
    return format_prediction_string(boxes, scores)

In [ ]:
import greatbarrierreef
env = greatbarrierreef.make_env()
iter_test = env.iter_test() 

for (pixel_array, df_pred) in iter_test:  # iterate through all test set images
    df_pred['annotations'] = predict(model, pixel_array)
    env.predict(df_pred)


In [ ]:
pd.read_csv('submission.csv')